In [16]:
#import libraries
import pandas as pd

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# load datasets
a = pd.read_csv('Answers.csv', usecols = ['Id', 'OwnerUserId', 'CreationDate', 'ParentId', 'Score', 'Body'], low_memory=False)
#print(a.shape)
#print(a.info())
#a.head()
q = pd.read_csv('Questions.csv', usecols = ['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title', 'Body'], low_memory=False)
#print(q.shape)
#print(q.info())
#q.head()
t = pd.read_csv('Tags.csv', usecols = ['Id', 'Tag'], low_memory=False)
#print(t.shape)
#print(t.info())
#t.head()

(1048575, 6)


,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


<H1>Clean the Data</H1>
We now have the data from kaggle loaded into three dataframes. There is much more data here than we need, as we are focusing on questions using the following programming languages: (java, c++, python). 

In order to do this, we will first create three new tag datasets with only java, python, and c++. We will then use these to remove all rows from the questions and answers that are not tagged with one of these three languages. 

In [23]:
# clean the data

# drop any rows where ID contains non-numerical values
q_clean = q[q['Id'].str.contains('\D') == False]
q_clean['Id'] = q_clean['Id'].astype(str).astype(int)
# drop any rows where Score contains non-numerical values
q_clean = q_clean[q_clean['Score'].str.contains('\D') == False]
q_clean['Score'] = q_clean['Score'].astype(str).astype(int)
#remove unwanted columns
q_clean.drop(["OwnerUserId"], axis=1, inplace=True)
print(q_clean.info())

<class 'pandas.core.frame.DataFrame'>
Index: 976367 entries, 0 to 1048574
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Id            976367 non-null  int32 
 1   CreationDate  976367 non-null  object
 2   ClosedDate    31217 non-null   object
 3   Score         976367 non-null  int32 
 4   Title         976367 non-null  object
 5   Body          976366 non-null  object
dtypes: int32(2), object(4)
memory usage: 44.7+ MB
None


In [26]:
#a.info()
#remove unwanted columns
a_clean = a
a_clean.drop(["Id", "OwnerUserId"], axis=1, inplace=True)
print(a_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   CreationDate  1048575 non-null  object
 1   ParentId      1048575 non-null  int64 
 2   Score         1048575 non-null  int64 
 3   Body          1048575 non-null  object
dtypes: int64(2), object(2)
memory usage: 32.0+ MB
None


In [28]:
t.info()
t['Tag'].value_counts(ascending=False).reset_index().head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Id      1048575 non-null  int64 
 1   Tag     1048236 non-null  object
dtypes: int64(1), object(1)
memory usage: 16.0+ MB


,Tag,count
0,c#,34979
1,java,29266
2,php,27837
3,javascript,26199
4,jquery,22814
5,android,22198
6,c++,14691
7,iphone,14482
8,python,13470
9,asp.net,13433


We can see that the languages we are interested in (java, python, c++) are all in the 15 most used tags within the dataset. We will now use the tags to remove any data we aren't interested in. 

Using the count of these tags, we know our cleaned tag dataset should have (29266+14691+13470) 57427 entries

In [8]:
t_java = pd.DataFrame(t[t.Tag == 'java'])
print("Java: ", t_java.shape)
print(t_java.head())

t_python = pd.DataFrame(t[t.Tag == 'python'])
print("Python: ", t_python.shape)
print(t_python.head())

t_cpp = pd.DataFrame(t[t.Tag == 'c++'])
print("C++: ", t_cpp.shape)
print(t_cpp.head())

Java:  (29266, 2)
        Id   Tag
127   4080  java
145   4630  java
211   7720  java
304  10980  java
346  11930  java
Python:  (13470, 2)
         Id     Tag
312   11060  python
503   17250  python
546   19030  python
905   31340  python
1027  34020  python
C++:  (14691, 2)
        Id  Tag
18     330  c++
107   3150  c++
112   3230  c++
216   7880  c++
302  10880  c++


Now we have the Ids for the three relevent tags in three separate datasets. We will use these to scrape the relevent rows from the questions and answers datasets. 

In [9]:
print(t_java.info())
print(q.info())

<class 'pandas.core.frame.DataFrame'>
Index: 29266 entries, 127 to 1048569
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      29266 non-null  int64 
 1   Tag     29266 non-null  object
dtypes: int64(1), object(1)
memory usage: 685.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Id            1045563 non-null  object
 1   OwnerUserId   1021828 non-null  object
 2   CreationDate  1034314 non-null  object
 3   ClosedDate    44685 non-null    object
 4   Score         1033835 non-null  object
 5   Title         1033783 non-null  object
 6   Body          1033747 non-null  object
dtypes: object(7)
memory usage: 56.0+ MB
None
